In [2]:
import os  # serve para manipular caminhos de diretórios e arquivos
import io  # lida com fluxos de arquivos
import numpy 
import pandas as pd  
from pandas import DataFrame  
from sklearn.feature_extraction.text import CountVectorizer  # transforma mensagens em valores numéricos
from sklearn.naive_bayes import MultinomialNB  # algoritmo responsável pelo aprendizado de máquina para classificação
from sklearn.model_selection import train_test_split  # para dividir os dados em treino e teste
from sklearn.metrics import accuracy_score  # para calcular a acurácia

# Função para ler os arquivos de e-mails
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):  # caminha pelos diretórios especificados
        for filename in filenames:  # itera sobre cada arquivo
            path = os.path.join(root, filename)  # pega o caminho completo do arquivo

            inBody = False  # identificar se estamos na parte do corpo do e-mail
            lines = []  # lista para armazenar as linhas do corpo do e-mail
            f = io.open(path, 'r', encoding='latin1')  # abre o arquivo 
            for line in f:
                if inBody:  # caso já esteja na parte do corpo, adicionamos as linhas
                    lines.append(line)
                elif line == '\n':  # uma linha em branco separa o cabeçalho do corpo
                    inBody = True
            f.close()  # fecha o arquivo
            message = '\n'.join(lines)  # junta as linhas do corpo em uma única string
            yield path, message  # retorna o caminho do arquivo e o conteúdo do e-mail

# Função para criar o DataFrame a partir dos arquivos em um diretório
def dataFrameFromDirectory(path, classification):
    rows = []  # lista para armazenar os dados de cada arquivo
    index = []  # lista para armazenar os nomes dos arquivos como índice
    for filename, message in readFiles(path):  # usa a função anterior para ler arquivos
        rows.append({'message': message, 'class': classification})  # adiciona uma linha com a mensagem e a classe ('spam' ou 'ham')
        index.append(filename)  # adiciona o nome do arquivo ao índice anteriormente citado
    return DataFrame(rows, index=index)  # retorna um DataFrame com os dados

# Cria um DataFrame vazio com colunas 'message' e 'class'
data = DataFrame({'message': [], 'class': []})  

# Concatena os DataFrames de spam e ham
data = pd.concat([data, dataFrameFromDirectory('/Users/heloamillergrassi/Downloads/MLCourse/emails/spam', 'spam')])
data = pd.concat([data, dataFrameFromDirectory('/Users/heloamillergrassi/Downloads/MLCourse/emails/ham', 'ham')])

# Criando o vetor de características a partir das mensagens
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['message'].values)  # Transforma as mensagens em uma matriz de contagem de palavras

# Definindo as classes (spam ou ham) como alvo
y = data['class'].values

# Dividindo os dados em conjunto de treino e conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criando o classificador Multinomial Naive Bayes
classifier = MultinomialNB()

# Treinando o classificador com os dados de treino
classifier.fit(X_train, y_train)

# Fazendo previsões com os dados de teste
y_pred = classifier.predict(X_test)

# Avaliando a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

# Mostrando a acurácia
print(f'Acurácia do modelo: {accuracy:.4f}')


Acurácia do modelo: 0.9544
